In [1]:
from gtts import gTTS
import playsound
import pymysql
import os
import subprocess

In [2]:
# localhost 개인 db접속
conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '0000',
    database = 'sound_db',
    charset = 'utf8'
)
cur = conn.cursor()

In [3]:
# 데이터베이스에서 docker_data의 데이터 유무 확인
def get_docker_data(di, ob):
    case = f'{di} {ob} 있습니다.'
    query = 'select docker_data from sound where txt=\"{}\";'.format(case)
    cur.execute(query)
    data = cur.fetchall()
    if data:
        docker_data = data[0][0]
        return docker_data, case
    else:
        return None, case

In [4]:
# result_case의 한글을 영어로 변환
def get_dieg_obeg(result_case):
    # 방향 정보 변환
    if result_case[0] == "왼쪽":
        dieg = "left"
    elif result_case[0] == "오른쪽":
        dieg = "right"

    # 객체 정보 변환
    if result_case[1] == "사람":
        obeg = "person"
    elif result_case[1] == "자전거":
        obeg = "bicycle"
    elif result_case[1] == "자동차":
        obeg = "car"
    elif result_case[1] == "오토바이":
        obeg = "motorcycle"
    elif result_case[1] == "버스":
        obeg = "bus"
    elif result_case[1] == "트럭":
        obeg = "truck"

    return dieg, obeg

In [5]:
# sound파일 검색
def docker_download_file(docker_data, download_path, case, dieg, obeg):
    if os.path.exists(download_path):
        return

    # 데이터가 있으면
    if docker_data:
        # 컨테이너 실행
        subprocess.run(["docker", "run", "--name", "blackboxsound", "sangho011/blackboxsound:blackbox"])
        # 컨테이너에서 파일 복사
        subprocess.run(["docker", "cp", f"blackboxsound:/blackboxsound/{dieg}{obeg}.mp3", "../blackboxsound"])
        # 컨테이너 삭제
        subprocess.run(["docker", "rm", "blackboxsound"])

    # 데이터가 없으면
    else:
        # gTTS로 sound파일 생성
        txt_path = case
        tts = gTTS(text=txt_path, lang='ko')
        tts.save(download_path)
        # 생성된 데이터의 docker_data정보 갱신
        query = 'UPDATE sound SET docker_data = \'O\', writer = \'han\', update_day = now() WHERE txt =\"{}\";'.format(case)
        cur.execute(query)
        conn.commit()

In [6]:
# 파일 다운로드 및 sound파일 재생
def download_file(dieg, obeg, docker_data, case):
    download_path=f"../blackboxsound/{dieg}{obeg}.mp3"
    docker_download_file(docker_data, download_path, case, dieg, obeg)
    playsound.playsound(download_path)

In [7]:
def play(result_case):
    di = result_case[0]
    ob = result_case[1]
    dieg, obeg = get_dieg_obeg(result_case)
    docker_data, case = get_docker_data(di, ob)
    download_file(dieg, obeg, docker_data, case)

In [8]:
result_case = [['오른쪽', '사람'],
                ['오른쪽', '사람'],
                ['오른쪽', '오토바이'],
                ['오른쪽', '사람'],
                ['왼쪽', '사람'],
                ['왼쪽', '자전거'],
                ['오른쪽', '오토바이']]

for case in result_case:
    play(case)

In [9]:
play(['오른쪽', '사람'])